In [ ]:
import numpy as np
import pandas as pd
# to display full text
pd.set_option('display.max_colwidth', -1)
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(r'graph_eligible_data_v2.csv', encoding = 'utf-8')

In [ ]:
df = df.drop('Unnamed: 0' , axis = 1)

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df_slice = df[['src_user_id', 'src_text','reply_user_id', 'reply_user_text']]

In [ ]:
import re
# function for cleaning data
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

df['clean_text_initiator'] = np.vectorize(remove_pattern)(df['src_text'], "@[\w]*")
df['clean_text_reply'] = np.vectorize(remove_pattern)(df['reply_user_text'], "@[\w]*")

In [ ]:
df['clean_text_initiator'] = df['clean_text_initiator'].str.replace("[^a-zA-Z#]", " ")
df['clean_text_reply'] = df['clean_text_reply'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
df['clean_text_initiator'] = df.clean_text_initiator.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
df['clean_text_reply'] = df.clean_text_reply.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))

In [ ]:
df.clean_text_initiator = df.clean_text_initiator.apply(lambda x: x.split())
df.clean_text_reply = df.clean_text_reply.apply(lambda x: x.split())

In [ ]:
from nltk.stem.porter import * 
stemmer = PorterStemmer() 
tokenized_tweet_text = df.clean_text_initiator.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet_text1 = df.clean_text_reply.apply(lambda x: [stemmer.stem(i) for i in x])

In [ ]:
for i in range(len(tokenized_tweet_text)):
    tokenized_tweet_text[i] = ' '.join(tokenized_tweet_text[i])    
df['clean_text_initiator'] = tokenized_tweet_text

for i in range(len(tokenized_tweet_text1)):
    tokenized_tweet_text1[i] = ' '.join(tokenized_tweet_text1[i])    
df['clean_text_reply'] = tokenized_tweet_text1

In [ ]:
df.head()

In [ ]:
df['src_verified_user'] = df['src_verified_user'].apply(lambda x: 1 if "True" else 0)

In [ ]:
df['reply_verified_user'] = df['reply_verified_user'].apply(lambda x: 1 if "True" else 0)

In [ ]:
 df['reply_label'] = 0

In [ ]:
df.shape

# Antonym and jaccard score

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
from nltk.corpus import wordnet

In [ ]:
def jaccard_similarity_score(str1, str2): 
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [ ]:
#threshold = 0.05
c = 1
for i in range (0, len(df)):
    print(i)
    initiator_label = df.iloc[i]['label']
    antonyms = []
    initiator_token = []
    reply_token = []
    antonyms_count = 0
    line_initiator = df.iloc[i]['clean_text_initiator']
    line_reply = df.iloc[i]['clean_text_reply']
    
    #tokens for initiators sentences
    #for l in line_initiator:
    initiator_token = word_tokenize(line_initiator)
    if (c == 1):
        print(initiator_token)
        c = c + 1
    
    #tokens for reply sentences
    #for l1 in line_reply:
    reply_token = word_tokenize(line_reply)
    
    #generate antonyms for initiator sentence
    for token in initiator_token:
        for syn in wordnet.synsets(token):
            for l2 in syn.lemmas():
                if l2.antonyms():
                    antonyms.append(l2.antonyms()[0].name())
    
    #comparing antonyms of initiator sentence with reply sentence
    for word in antonyms:
        for compared_word in reply_token:
            if (word == compared_word):
                print("word ",word, " compared ", compared_word)
                antonyms_count = antonyms_count + 1
            
    #score calculation
    #antonym_score = (antonyms_count) / (len(initiator_token) + len(reply_token))
    #print(antonym_score)
    
    jaccard_score = jaccard_similarity_score(df.iloc[i]['clean_text_initiator'], df.iloc[i]['clean_text_reply'])
    
    count = 0
    #assigning labels to reply nodes
    if (antonyms_count >= 1 or jaccard_score < 0.5):
        print(count)
        count = count + 1
        df.loc[i,'reply_label'] = 1 - initiator_label
        #print(" reply label for opposites: ", df.loc[i,'reply_label'])
        #print("if:", df.iloc[i]['label'], " reply: ", )
    else:
        df.loc[i,'reply_label'] = initiator_label
    

In [ ]:
df['reply_label'].describe()

In [ ]:
count_0 = 0
count_1 = 0
for i in range(0, len(df)):
    if (df.iloc[i]['reply_label'] == 0):
        count_0 = count_0 + 1
    elif (df.iloc[i]['reply_label'] == 1):
        count_1 = count_1 + 1
print("0: ", count_0)
print("1: ", count_1)
percent_0 = (count_0) / (len(df) ) * 100
print(percent_0)

percent_1 = (count_1) / (len(df) ) * 100
print(percent_1)

In [ ]:
sum(df['reply_label'].apply(lambda x: x if 1 else 0))

In [ ]:
df.head()

# not to be executed

In [ ]:
df.to_csv('charlie_replynode_labelling.csv')

In [ ]:
df = pd.read_csv('charlie_replynode_labelling.csv')

In [ ]:
df.nunique()

In [ ]:
len(df[df['src_user_id'] == df['reply_user_id']])

In [ ]:
df.info()

In [ ]:
df = df.drop('Unnamed: 0' , axis = 1)

In [ ]:
#18992 rows

In [ ]:
df.shape[0]

In [ ]:
indices = []
for i in range(0, df.shape[0]):
    indices.append(i)
len(indices)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
columns = ['user_id', 'text', 'favorites_count', 'verified', 'followers_count', 'label']

In [ ]:
df_graph = pd.DataFrame(data = np.zeros((df.shape[0], 6)), index = indices, columns = columns)

In [ ]:
list_of_initiators = df['src_user_id'].unique()

In [ ]:
#adding data in linear fashion asll user ids in one column for src_ids here
for i in range (0, len(list_of_initiators)):
    print(i)
    count = 0
    for j in range(0, len(df)):
        if ((list_of_initiators[i] == df.iloc[j]['src_user_id']) & (count == 0)):
            count = count + 1
            df_graph.loc[i,'user_id'] = df.iloc[j]['src_user_id']
            df_graph.loc[i,'text'] = df.iloc[j]['clean_text_initiator']
            df_graph.loc[i,'favorites_count'] = df.iloc[j]['src_favorites_count']
            df_graph.loc[i,'followers_count'] = df.iloc[j]['src_followers_count']
            df_graph.loc[i,'verified'] = df.iloc[j]['src_verified_user']
            df_graph.loc[i,'label'] = df.iloc[j]['label']


In [ ]:
df_graph.iloc[1005]

In [ ]:
list_of_replies = df['reply_user_id'].unique()

# not to be executed

In [ ]:
df_graph.to_csv('graph-data.csv')

In [ ]:
df_graph = pd.read_csv('graph-data.csv')

In [ ]:
df_graph = df_graph.drop('Unnamed: 0' , axis = 1)

In [ ]:
df_graph['user_id'].nunique()

In [ ]:
df_graph.iloc[1004]

In [ ]:
type(df.iloc[0]['clean_text_reply'])

In [ ]:
len(list_of_replies)

In [ ]:
#adding data in linear fashion as all user ids in one column for reply ids here
k = len(list_of_initiators)
for i in range (0, len(list_of_replies)):
    k = k + 1
    print(i)
    count = 0
    text = ''
    for j in range(0, len(df)):
        if (list_of_replies[i] == df.iloc[j]['reply_user_id']):
            if (count == 0):
                df_graph.loc[k,'user_id'] = df.iloc[j]['reply_user_id']
                df_graph.loc[k,'favorites_count'] = df.iloc[j]['reply_favorites_count']
                df_graph.loc[k,'followers_count'] = df.iloc[j]['reply_followers_count']
                df_graph.loc[k,'verified'] = df.iloc[j]['reply_verified_user']
                df_graph.loc[k,'label'] = df.iloc[j]['reply_label']
                count = count + 1
                    
            text = text + str(df.iloc[j]['clean_text_reply'])

    df_graph.loc[k,'text'] = text

In [ ]:
#adding data in linear fashion asll user ids in one column for reply ids here
k = 17991
for i in range (0, 1006):
    k = k + 1
    print(i)
    count = 0
    text = ''
    for j in range(0, len(df)):
        if (list_of_replies[i] == df.iloc[j]['reply_user_id']):
            if (count == 0):
                df_graph.loc[k,'user_id'] = df.iloc[j]['reply_user_id']
                df_graph.loc[k,'favorites_count'] = df.iloc[j]['reply_favorites_count']
                df_graph.loc[k,'followers_count'] = df.iloc[j]['reply_followers_count']
                df_graph.loc[k,'verified'] = df.iloc[j]['reply_verified_user']
                df_graph.loc[k,'label'] = df.iloc[j]['reply_label']
                count = count + 1
                    
            text = text + str(df.iloc[j]['clean_text_reply'])

    df_graph.loc[k,'text'] = text

In [ ]:
df_graph.shape

In [ ]:
df_graph.describe()

df_graph.to_csv('graph-datav2.csv')

df_graph.to_csv('graph-datav3.csv')

In [ ]:
#converting labels to user suspicious score (for src_ids)
for i in range (0, len(list_of_initiators)):
    print(i)
    label_0 = 0
    label_1 = 0
    for j in range(0, len(df)):
            if (list_of_initiators[i] == df.iloc[j]['src_user_id']):
                if (df.iloc[j]['label'] == 0):
                    label_0 = label_0 + 1
                elif (df.iloc[j]['label'] == 1):
                    label_1 = label_1 + 1
    
    suspicious_user_intensity = round(((label_1)/(label_1 + label_0)), 2)            
    print("suspicious_user_intensity: ", suspicious_user_intensity)
    df_graph.loc[i,'label'] = suspicious_user_intensity

In [ ]:
df_graph.iloc[553]

In [ ]:
len(list_of_initiators)

In [ ]:
#converting labels to user suspicious score (for reply_ids)
k = -1
for i in range (len(list_of_initiators), len(df_graph)):
    k = k + 1
    print(i)
    label_0 = 0
    label_1 = 0
    for j in range(0, len(df)):
            if (list_of_replies[k] == df.iloc[j]['reply_user_id']):
                if (df.iloc[j]['reply_label'] == 0):
                    label_0 = label_0 + 1
                elif (df.iloc[j]['reply_label'] == 1):
                    label_1 = label_1 + 1
    
    suspicious_user_intensity = round(((label_1)/(label_1 + label_0)), 2)            
    print("suspicious_user_intensity: ", suspicious_user_intensity)
    df_graph.loc[i,'label'] = suspicious_user_intensity

In [ ]:
df_graph.shape

In [ ]:
df_graph['label'].unique()

In [ ]:
print(k)

In [ ]:
len(list_of_replies)

In [ ]:
len(list_of_initiators)

In [ ]:
len(df_graph)

In [ ]:
df_graph.iloc[11234]

In [ ]:
reduce_df = df_graph[:11234]

In [ ]:
reduce_df.shape

## not to be executed

In [ ]:
df_graph.to_csv('graph-datav4.csv')

In [ ]:
reduce_df.to_csv('graph-datav4.csv')

In [ ]:
df_graph = pd.read_csv('graph-datav4.csv')

In [ ]:
df_graph['user_id'].nunique()

In [ ]:
reduce_df['user_id'].nunique()

In [ ]:
df_graph.describe()

In [ ]:
count_0 = 0
count_1 = 0
for i in range(0, len(reduce_df)):
    if (reduce_df.iloc[i]['label'] == 0):
        count_0 = count_0 + 1
    elif (reduce_df.iloc[i]['label'] == 1):
        count_1 = count_1 + 1
print("0: ", count_0)
print("1: ", count_1)
percent_0 = (count_0) / (len(reduce_df) ) * 100
print(percent_0)

percent_1 = (count_1) / (len(reduce_df) ) * 100
print(percent_1)

In [ ]:
reduce_df['label'].unique()

In [ ]:
reduce_df.shape

In [ ]:
df_graph.shape

# Manual Word2Vec

## Word Embeddings using gensim

In [ ]:
import gensim

In [ ]:
model = gensim.models.Word2Vec(sentences = text, size = 100, window = 2, workers = 3, min_count = 1)

In [ ]:
words = list(model.wv.vocab)
print('Vocab size', len(words))

In [ ]:
words

## Most Similar Words

In [ ]:
model.wv.most_similar('terror')

## Save vectors

In [ ]:
file = 'twitter_charlie_manual_word2vec.txt'

In [ ]:
model.wv.save_word2vec_format(file,binary = False)

In [ ]:
import os
emb = {}
f = open(os.path.join('', 'twitter_graph_charlie_manual_word2vec.txt'), encoding = 'utf-8')
for line in f:
    values = line.split()
    word = values[0]
    value = np.asarray(values[1:])
    emb[word] = value
f.close()

In [ ]:
emb['well']

In [ ]:
token = Tokenizer()
token.fit_on_texts(text)
seq = token.texts_to_sequences(text)
index = token.word_index
print('unique no of tokens ', index)
padding = pad_sequences(seq, maxlen = max_length)
rumour = df_graph['label'].values
print('Shape of tweet ',padding.shape)
print('Shape of rumour ', rumour.shape)

In [ ]:
padding[13]

In [ ]:
rumour

In [ ]:
n = len(index) + 1
embed_mat = np.zeros((n, 100))
for s,values in index.items():
    print(s,values)
    if values>n:
        continue
    embed_vec = emb.get(s)
    if embed_vec is not None:
        embed_mat[values] = embed_vec
print(n)

In [ ]:
embed_mat.shape

In [ ]:
len(index)

In [ ]:
sorted_words = sorted(index)
print(sorted_words)

In [ ]:
index.keys()

## Map words with their vectors

In [ ]:
features = {}

In [ ]:
d = np.arange(0, len(padding))

In [ ]:
data = np.zeros((0, len(df_graph)))

In [ ]:
feat_df = pd.DataFrame(index = d, data = data, columns = sorted_words)
print(feat_df)

In [ ]:
feat_df.shape

In [ ]:
padding[4]

In [ ]:
embed_mat[2].mean()

In [ ]:
feat_df.iloc[1][1] == 0.0

In [ ]:
#combining features with average
for row in range(0, len(padding)):
    print(row)
    for number in padding[row]:
        for key,value in index.items():
            if number == value:
                mean = embed_mat[number].mean()
                if (feat_df.iloc[row][key] == 0.0):
                    feat_df.iloc[row][key] = mean
                else:
                    feat_df.iloc[row][key] = feat_df.iloc[row][key] + mean

In [ ]:
feat_df.to_csv('features of word2vec embeddings.csv')

In [ ]:
feat_df=feat_df.fillna(0)

In [ ]:
feat_df.shape

## Combining Features into 1 dataframe

In [ ]:
df_graph.drop('clean_text', axis = 1, inplace = True)

In [ ]:
df_graph.head()

In [ ]:
X_new = pd.concat([feat_df, df_graph], axis=1)

In [ ]:
X_new.shape

X_new.to_csv('charlie_combined_word2vec_features_with_original_features.csv')

# Graph

In [ ]:
G=nx.Graph()

for i in range(0, len(df)):
    G.add_edge(df.iloc[i]['src_user_id'], df.iloc[i]['reply_user_id'], weight = df.iloc[i]['weight'])

nx.degree(G)

In [ ]:
print(nx.average_clustering(G))
nx.density(G)

In [ ]:
nx.is_connected(G)

**Adjacency matrix**

In [ ]:
A = nx.adjacency_matrix(G)

In [ ]:
A.shape